# RAG with arXiv Papers

Retrieval-augmented generation (RAG) provides large language models additional helpful information in a prompt that is retrieved when a user submits a query.

This guide uses Atlas as a data layer for retrieval, followed by LLM inference using the information queried from our Atlas Dataset via vector search.

## Setup

In [ ]:
!pip install nomic

In [2]:
!nomic login nk-rKYMrmxBV5F7B5FK-XkFPGDh3G91b_F0eA9ZtVR-BLY

## Create Atlas Dataset

Let's start with a collection with PDFs and chunk them into snippets to be fetched for retrieval.

For this example, we will download PDFs from the open-access paper repository arXiv.

Make sure these libraries are installed to your python environment:

In [ ]:
!pip install PyPDF2 requests

Now, we can download PDFs from arxiv and prepare them as a dataset of paper chunks. This way, we will be able to retrieve specific relevant snippets from papers (instead of entire papers) later on when running RAG with an LLM.

In [ ]:
import io
import requests
import PyPDF2

ARXIV_PAPERS = [
    ("1706.03762", "Attention Is All You Need"),
    ("1512.03385", "Deep Residual Learning"),
    ("1810.04805", "BERT"),
    ("2005.14165", "GPT-3"),
    ("1412.6980", "Adam"),
    ("1406.2661", "GANs"),
    ("1505.04597", "U-Net"),
    ("2204.06125", "DALL-E 2"),
    ("2112.10752", "Stable Diffusion")
]

# load text from PDFs into a list of dicts called 'data'
# each containing a text chunk plus metadata
data = []
text_chunk_size = 500
for paper_id, title in ARXIV_PAPERS:
    print("Downloading", title)
    pdf = PyPDF2.PdfReader(io.BytesIO(requests.get(f"https://arxiv.org/pdf/{paper_id}.pdf").content))    
    for page_num, page in enumerate(pdf.pages):
        text = page.extract_text()
        for i in range(0, len(text), text_chunk_size):
            chunk = text[i:i + text_chunk_size].strip()
            data.append({
                "filename": f"arxiv_{paper_id}.pdf",
                "title": title,
                "page_number": page_num + 1,
                "text": chunk
            })

Let's take a look at the data to make sure it looks alright:

In [ ]:
data[50]

## Upload Dataset to Atlas

In [ ]:
from nomic import atlas

atlas_dataset = atlas.map_data(
    data=data,
    indexed_field="text",
    identifier="rag-pdf-dataset"
)

You'll get an email when your data map is built!

## Retrieval Over Your Data Map

Now that you have an Atlas Dataset, you can use it as the data layer for your application. Our vector search endpoint returns the k-most semantically similar items from your Atlas Dataset based on a query.

This example function makes an API call to Atlas's vector search endpoint:

In [9]:
import requests
import os

def retrieve_context(query: str, projection_id: str, k: int, fields: list[str]) -> list:
    """Retrieve semantically similar items from Atlas based on a query."""
    response = requests.post(
        'https://api-atlas.nomic.ai/v1/query/topk',
        headers={'Authorization': f'Bearer {os.environ.get("NOMIC_API_KEY")}'},
        json={
            'projection_id': projection_id,
            'k': k,
            'query': query,
            'fields': fields
        }
    )
    return response.json()['data']

The important parameters for this endpoint are:

• query: The text query to search against

• k: Number of similar items to return

• fields: List of fields from your dataset to return in the response

• projection_id: The unique identifier for your Atlas map (you can find it on the dataset page for any dataset on your [Atlas Dashboard](https://docs.nomic.ai/atlas/introduction/quick-start#atlas-dashboard)

For example, here is the output for retrieve_content on the previously created dataset of ML papers:

In [13]:
# update this with the projection ID
# on the dataset page for your new Atlas map
my_map_projection_id = "a3392b83-a135-4e4c-addb-a8b71ca59f5c"

In [14]:
retrieved_data = retrieve_context(
    "What is attention in deep learning?", # Query
    my_map_projection_id,  # Your Atlas projection ID
    3,  # Return top 3 results
    ["title", "text"]  # Fields to retrieve
)

Let's take a look at the retrieved data to make sure it looks alright:

In [ ]:
retrieved_data

# End-to-End RAG with the Data Map

With a retrieval function for our data map, we can now perform end-to-end RAG.

## Install Ollama

We will run LLM generation using Ollama, which you can download here.

Once ollama is installed to your machine, install Llama 3.2 1b at your terminal with 

`ollama pull llama3.2:1b` 

## Full Code Example

We now add an additional function generate_response which sends the result of an Atlas Dataset retrieval and sends it to an Ollama LLM (at the default Ollama endpoint http://localhost:11434/api/chat).

The response is then assembled into a single string for the user:

In [15]:
import json
import requests
import os

def retrieve_context(query: str, projection_id: str, k: int, fields: list[str]) -> list:
    """Retrieve semantically similar items from Atlas based on a query."""
    response = requests.post(
        'https://api-atlas.nomic.ai/v1/query/topk',
        headers={'Authorization': f'Bearer {os.environ.get("NOMIC_API_KEY")}'},
        json={
            'projection_id': projection_id,
            'k': k,
            'query': query,
            'fields': fields
        }
    )
    return response.json()['data']


def generate_response(retrieved_data: list, query: str) -> str:
    """Generate a response using Ollama based on retrieved data and query."""
    response = requests.post(
        'http://localhost:11434/api/chat',
        json={
            'model': 'llama3.2:1b',
            'messages': [{
                'role': 'user',
                'content': f"Context:\n{retrieved_data}\n\nQuestion: {query}"
            }]
        }
    )
    return ''.join(
        json.loads(line)['message']['content']
        for line in response.text.strip().split('\n')
        if line and not json.loads(line).get('done', False)
    )

query = "What is attention in deep learning?"

retrieved_data = retrieve_context(
    query,
    my_map_projection_id,  # Your Atlas projection ID
    3,  # Return top 3 results
    ["title", "text"]  # Fields to retrieve
)

response = generate_response(retrieved_data, query)

Now let's print the RAG response:

In [ ]:
print(f"Q: {query}\nA: {response}")